In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv(r'C:\Users\rahil\Desktop\Nexus-Edge\data.csv')
df

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.00,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.00,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.00,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.00,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.00,canceled,14,US,1283.00,1283.00,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.00,successful,224,US,52375.00,52375.00,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.00,successful,16,US,1205.00,1205.00,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.00,failed,40,US,453.00,453.00,25000.00
8,1000034518,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,USD,2014-05-29,125000.0,2014-04-24 18:14:43,8233.00,canceled,58,US,8233.00,8233.00,125000.00
9,100004195,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,USD,2014-08-10,65000.0,2014-07-11 21:55:48,6240.57,canceled,43,US,6240.57,6240.57,65000.00


In [2]:
df=df.drop('ID',axis=1)
df=df.drop('currency',axis=1)
df=df.drop('usd pledged',axis=1)
df=df.drop('goal',axis=1)
x=df
x.shape

(378661, 11)

In [3]:
x.insert(11,'goal',(x['usd_goal_real']).astype(int))
x=x.drop('usd_goal_real',axis=1)
x
y=x['state']
y


0             failed
1             failed
2             failed
3             failed
4           canceled
5         successful
6         successful
7             failed
8           canceled
9           canceled
10            failed
11        successful
12            failed
13            failed
14        successful
15            failed
16            failed
17            failed
18        successful
19            failed
20        successful
21            failed
22            failed
23            failed
24        successful
25        successful
26            failed
27        successful
28        successful
29        successful
             ...    
378631    successful
378632        failed
378633        failed
378634        failed
378635        failed
378636    successful
378637    successful
378638        failed
378639    successful
378640      canceled
378641        failed
378642    successful
378643      canceled
378644    successful
378645        failed
378646    successful
378647       

In [20]:
x=x.drop('country',axis=1)
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))




from matplotlib.pyplot import figure, show
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from sklearn import metrics
from scipy.stats import zscore
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
   
state=encode_text_index(x,'state')
num_classes = len(state)
    
#x,y=to_xy(x,'state')

ValueError: labels ['country'] not contained in axis

In [6]:
from datetime import datetime
#s1 = '10:33:26'
#s2 = '11:15:49' # for example
x['deadline']=pd.to_datetime(x['deadline'])
x['launched']=pd.to_datetime(x['launched'])

FMT_1 = '%Y-%m-%d'
FMT_2 = '%Y-%m-%d %H:%M:%S'
#tdelta = datetime.strptime(x['deadline'], FMT_1) - datetime.strptime(x['launched'], FMT_2)
x['deadline']-x['launched']
x.insert(4,'time_span',(x['deadline']-x['launched']).astype(datetime))


In [7]:
x=x.drop('deadline',axis=1)
x=x.drop('launched',axis=1)
x

,name,category,main_category,time_span,pledged,state,backers,usd_pledged_real,goal
0,The Songs of Adelaide & Abullah,Poetry,Publishing,58 days 11:47:32,0.00,1,0,0.00,1533
1,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,59 days 19:16:03,2421.00,1,15,2421.00,30000
2,Where is Hank?,Narrative Film,Film & Video,44 days 23:39:10,220.00,1,3,220.00,45000
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,29 days 20:35:49,1.00,1,1,1.00,5000
4,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,55 days 15:24:57,1283.00,0,14,1283.00,19500
5,Monarch Espresso Bar,Restaurants,Food,34 days 10:21:33,52375.00,3,224,52375.00,50000
6,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,19 days 05:29:16,1205.00,3,16,1205.00,1000
7,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,44 days 03:54:48,453.00,1,40,453.00,25000
8,SPIN - Premium Retractable In-Ear Headphones w...,Product Design,Design,34 days 05:45:17,8233.00,0,58,8233.00,125000
9,STUDIO IN THE SKY - A Documentary Feature Film...,Documentary,Film & Video,29 days 02:04:12,6240.57,0,43,6240.57,65000


In [8]:
x=x.drop('name',axis=1)
x=x.drop('category',axis=1)
x=x.drop('main_category',axis=1)
x


,time_span,pledged,state,backers,usd_pledged_real,goal
0,58 days 11:47:32,0.00,1,0,0.00,1533
1,59 days 19:16:03,2421.00,1,15,2421.00,30000
2,44 days 23:39:10,220.00,1,3,220.00,45000
3,29 days 20:35:49,1.00,1,1,1.00,5000
4,55 days 15:24:57,1283.00,0,14,1283.00,19500
5,34 days 10:21:33,52375.00,3,224,52375.00,50000
6,19 days 05:29:16,1205.00,3,16,1205.00,1000
7,44 days 03:54:48,453.00,1,40,453.00,25000
8,34 days 05:45:17,8233.00,0,58,8233.00,125000
9,29 days 02:04:12,6240.57,0,43,6240.57,65000


In [9]:
y=x['state']
y

0         1
1         1
2         1
3         1
4         0
5         3
6         3
7         1
8         0
9         0
10        1
11        3
12        1
13        1
14        3
15        1
16        1
17        1
18        3
19        1
20        3
21        1
22        1
23        1
24        3
25        3
26        1
27        3
28        3
29        3
         ..
378631    3
378632    1
378633    1
378634    1
378635    1
378636    3
378637    3
378638    1
378639    3
378640    0
378641    1
378642    3
378643    0
378644    3
378645    1
378646    3
378647    1
378648    0
378649    1
378650    1
378651    3
378652    3
378653    1
378654    1
378655    0
378656    0
378657    1
378658    1
378659    1
378660    1
Name: state, Length: 378661, dtype: int64

In [10]:
x['time_span']=pd.to_timedelta(x['time_span'])

In [11]:
#x['time_span'].days.astype(int)
days = x['time_span'].astype('timedelta64[D]')
days

0         58.0
1         59.0
2         44.0
3         29.0
4         55.0
5         34.0
6         19.0
7         44.0
8         34.0
9         29.0
10        29.0
11        29.0
12        29.0
13        44.0
14        27.0
15        29.0
16        29.0
17        29.0
18        14.0
19        29.0
20        34.0
21        29.0
22        29.0
23        29.0
24        19.0
25        29.0
26        29.0
27        29.0
28        29.0
29        29.0
          ... 
378631    29.0
378632    29.0
378633    29.0
378634    60.0
378635    29.0
378636    29.0
378637    44.0
378638    30.0
378639    29.0
378640    33.0
378641    59.0
378642    26.0
378643    20.0
378644    21.0
378645    29.0
378646    42.0
378647    29.0
378648    29.0
378649    59.0
378650    37.0
378651    29.0
378652    29.0
378653    29.0
378654    29.0
378655    44.0
378656    29.0
378657    26.0
378658    45.0
378659    30.0
378660    27.0
Name: time_span, Length: 378661, dtype: float64

In [12]:
#x.insert(4,'days',days)
x=x.drop('time_span',axis=1)

In [13]:
x.insert(4,'days',days)
x

,pledged,state,backers,usd_pledged_real,days,goal
0,0.00,1,0,0.00,58.0,1533
1,2421.00,1,15,2421.00,59.0,30000
2,220.00,1,3,220.00,44.0,45000
3,1.00,1,1,1.00,29.0,5000
4,1283.00,0,14,1283.00,55.0,19500
5,52375.00,3,224,52375.00,34.0,50000
6,1205.00,3,16,1205.00,19.0,1000
7,453.00,1,40,453.00,44.0,25000
8,8233.00,0,58,8233.00,34.0,125000
9,6240.57,0,43,6240.57,29.0,65000


In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=42)
svm = SVC(kernel='rbf', C=1).fit(x_train[:40500], y_train[:40500])

score=np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))
svm.score(x_test[:10000], y_test[:10000])

Score (RMSE): 1.3103252390372293


0.53539999999999999

In [ ]:
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

pred=svm.predict(x_test)
chart_regression(pred,y_test)